# Imports

In [16]:
import pandas as pd
import numpy as np

In [17]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pylab as plt

# Functions

In [18]:
def clean_data():
    
    global db1
    db1 = db1[(db1.num_infections > 0)].reset_index()
    db1 = db1.drop(columns = ['index'], axis = 1)
    
    db1['date'] = pd.to_datetime(db1.date)
    
   
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Valenciana' if x == 'Valenciana, Comunidad' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad de Madrid' if x == 'Madrid, Comunidad de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Región de de Murcia' if x == 'Murcia, Región de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Foral de Navarra' if x == 'Navarra, Comunidad Foral de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Principado de Asturias' if x == 'Asturias, Principado de' else x)
    
    
    db1['province'] = db1.province.apply(lambda x: 'Alicante' if x == 'Alicante/Alacant' else x)
    db1['province'] = db1.province.apply(lambda x: 'Castellón' if x == 'Castellón/Castelló' else x)
    db1['province'] = db1.province.apply(lambda x: 'Araba' if x == 'Araba/Álava' else x)
    
    db1['sex'] = db1.sex.apply(lambda x: 'UKNOWN' if x == 'NC' else x)
    
    db1['age_interval'] = db1.age_interval.apply(lambda x: 'UKNOWN' if x == 'NC' else x)

In [19]:
def cumulative(dataframe):
    
    cumu_num_infections = dataframe.num_infections.cumsum()
    cumu_num_hosp = dataframe.num_hosp.cumsum()
    cumu_num_uci = dataframe.num_uci.cumsum()
    cumu_num_dead = dataframe.num_dead.cumsum()

    cumulative = pd.DataFrame({'cumu_num_infections': cumu_num_infections, 'cumu_num_hosp': cumu_num_hosp, 
                               'cumu_num_uci': cumu_num_uci, 'cumu_num_dead': cumu_num_dead})
    dataframe = pd.concat([dataframe, cumulative], axis = 1)
    return dataframe

In [20]:
def freq_rel(dataframe):
    
    dataframe['Number of infection (%)'] = [round(i/ dataframe['num_infections'].sum(),3)*100 for i in dataframe['num_infections']]
    dataframe['Number of hospitalisation (%)'] = [round(i/ dataframe['num_hosp'].sum(),3)*100 for i in dataframe['num_hosp']]
    dataframe['Number of Intensive Care Unit (%)'] = [round(i/ dataframe['num_uci'].sum(),3)*100 for i in dataframe['num_uci']]
    dataframe['Number of deaths (%)'] = [round(i/ dataframe['num_dead'].sum(),3)*100 for i in dataframe['num_dead']]

In [21]:
def organise(column):
    
    name = db1.groupby(column).sum()
    freq_rel(name)
    
    return name

In [22]:
def mov_7_ave(dataframe):
    dataframe['ave_7_num_infections'] = dataframe.iloc[:,1].rolling(window=7).mean()
    dataframe['ave_7_num_hosp'] = dataframe.iloc[:,2].rolling(window=7).mean()
    dataframe['ave_7_num_uci'] = dataframe.iloc[:,3].rolling(window=7).mean()
    dataframe['ave_7_num_dead'] = dataframe.iloc[:,4].rolling(window=7).mean()
    
    return dataframe

# Data Base

In [23]:
db1 =pd.read_csv('/content/covid-19-sample.csv') 

In [26]:
db1

,date,autonomous_region,province,sex,age_interval,num_infections,num_hosp,num_uci,num_dead
0,2020-03-02,Comunidad Valenciana,Alicante,H,60-69,1,1,0,0
1,2020-03-02,Comunidad Valenciana,Alicante,H,70-79,1,1,0,0
2,2020-03-02,Comunidad Valenciana,Alicante,M,30-39,1,0,0,0
3,2020-03-02,Comunidad Valenciana,Alicante,M,70-79,1,0,0,0
4,2020-03-02,Comunidad Valenciana,Alicante,M,80+,1,0,0,0
...,...,...,...,...,...,...,...,...,...
300931,2021-06-10,Castilla y León,Zamora,H,50-59,2,0,0,0
300932,2021-06-10,Castilla y León,Zamora,M,10-19,3,0,0,0
300933,2021-06-10,Castilla y León,Zamora,M,20-29,2,0,0,0
300934,2021-06-10,Castilla y León,Zamora,M,40-49,1,0,0,0


In [25]:
clean_data()

In [27]:
bydate = db1.groupby('date').sum().reset_index()
bydate = cumulative(bydate)
bydate = mov_7_ave(bydate)

In [33]:
bydate.head()

,date,num_infections,num_hosp,num_uci,num_dead,cumu_num_infections,cumu_num_hosp,cumu_num_uci,cumu_num_dead,ave_7_num_infections,ave_7_num_hosp,ave_7_num_uci,ave_7_num_dead
0,2020-01-18,1,1,0,0,1,1,0,0,NaN,NaN,NaN,NaN
1,2020-01-31,1,0,0,0,2,1,0,0,NaN,NaN,NaN,NaN
2,2020-02-09,1,1,0,0,3,2,0,0,NaN,NaN,NaN,NaN
3,2020-02-12,3,0,0,0,6,2,0,0,NaN,NaN,NaN,NaN
4,2020-02-21,1,0,0,0,7,2,0,0,NaN,NaN,NaN,NaN


# ARMA

In [34]:
from statsmodels.tsa.arima_model import ARMA

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [35]:
from sklearn.metrics import mean_squared_error as mse

# num_infections

In [36]:
infections = bydate[['date', 'ave_7_num_infections']].set_index('date')

In [37]:
infections = infections[6:]

In [38]:
infections

,ave_7_num_infections
date,
2020-02-23,1.428571
2020-02-24,1.571429
2020-02-25,1.571429
2020-02-26,2.428571
2020-02-27,5.285714
...,...
2021-06-06,4015.142857
2021-06-07,3911.142857
2021-06-08,3761.857143


# 1 day

In [39]:
train, test = infections[:-1], infections[-1:]

In [40]:
train

,ave_7_num_infections
date,
2020-02-23,1.428571
2020-02-24,1.571429
2020-02-25,1.571429
2020-02-26,2.428571
2020-02-27,5.285714
...,...
2021-06-05,4048.428571
2021-06-06,4015.142857
2021-06-07,3911.142857


In [41]:
test

,ave_7_num_infections
date,
2021-06-10,3504.714286


In [48]:
model.predict(len(train), len(infections)-1)

2021-06-10    3470.281124
Freq: D, dtype: float64

In [53]:
res

[(1, 1, 18.90747564270123),
 (1, 2, 12.645228909246725),
 (1, 3, 16.02402959087658),
 (1, 4, 17.424846259546484),
 (1, 5, 13.158815228540334),
 (1, 6, 10.534746179435388),
 (1, 7, 6.020405179261616),
 (2, 1, 3.8629991798852643),
 (2, 2, 2.5379584929751218),
 (2, 3, 1.5548487226483303),
 (2, 4, 2.504379702289043),
 (2, 5, 8.292562055577491),
 (2, 6, 34.63647317724053),
 (2, 7, 20.346714899237213),
 (3, 1, 2.962543241550975),
 (3, 2, 3.731526111430412),
 (3, 3, 9.857697807683962),
 (3, 4, 18.916415229525683),
 (3, 5, 16.35032047720324),
 (3, 6, 36.29986501237863),
 (3, 7, 25.36505903340276),
 (4, 1, 3.7822478493039853),
 (4, 2, 0.872629242644507),
 (4, 3, 10.865939081843862),
 (4, 4, 14.671905625944873),
 (4, 5, 18.741172717896916),
 (4, 6, 26.842763727696365),
 (4, 7, 19.86533352639799)]

In [52]:
res=[]
for p in range(1, 5):
  for q in range(1, 8):
    model=ARMA(train, order = (p, q)).fit(disp=False)
    res.append((p, q, mse(model.predict(len(train), len(infections)-1), test)**0.5))

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packa

In [54]:
error = [e for p, q, e in res]
best = [res[i] for i, e in enumerate(error) if e == min(error)]

best


[(4, 2, 0.872629242644507)]

# 3 days

In [115]:
train, test = infections[:-3], infections[-3:]

In [124]:
res=[]
for p in range(1, 9):
  for q in range(1, 15):
    try:
      model=ARMA(train, order = (p, q)).fit(disp=False)
      res.append((p, q, mse(model.predict(len(train), len(infections)-1), test)**0.5))
    except:
      continue

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packa

In [104]:
model.predict(len(train), len(infections)-1), test

(2021-06-08    3848.515346
 2021-06-09    3800.551868
 2021-06-10    3768.123391
 Freq: D, dtype: float64,             ave_7_num_infections
 date                            
 2021-06-08           3761.857143
 2021-06-09           3593.714286
 2021-06-10           3504.714286)

In [125]:
error = [e for p, q, e in res]
best = [res[i] for i, e in enumerate(error) if e == min(error)]

best

[(4, 12, 103.05715108983061)]

In [123]:
error = [e for p, q, e in res]
best = [res[i] for i, e in enumerate(error) if e == min(error)]

best

[(4, 9, 176.02166986878916)]

In [98]:
error = [e for p, q, e in res]
best = [res[i] for i, e in enumerate(error) if e == min(error)]

best

[(2, 7, 187.92415291857648)]

## 5 days

In [105]:
train, test = infections[:-5], infections[-5:]

In [114]:
model=ARMA(train, order = (8, 7)).fit(disp=False)
mse(model.predict(len(train), len(infections)-1), test)**0.5

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


239.5150638824939

In [ ]:
res=[]
for p in range(1, 9):
  for q in range(1, 10):
    try:
      model=ARMA(train, order = (p, q)).fit(disp=False)
      res.append((p, q, mse(model.predict(len(train), len(infections)-1), test)**0.5))
    except:
      continue

In [107]:
mse(model.predict(len(train), len(infections)-1), test)**0.5

333.239000581947

In [ ]:
res=[]
for p in range(1, 5):
  for q in range(1, 8):
    model=ARMA(train, order = (p, q)).fit(disp=False)
    res.append((p, q, mse(model.predict(len(train), len(infections)-1), test)**0.5))

In [112]:
for q in range(1, 8):
  print(q)

1
2
3
4
5
6
7


# num_hosp

In [55]:
bydate.head()

,date,num_infections,num_hosp,num_uci,num_dead,cumu_num_infections,cumu_num_hosp,cumu_num_uci,cumu_num_dead,ave_7_num_infections,ave_7_num_hosp,ave_7_num_uci,ave_7_num_dead
0,2020-01-18,1,1,0,0,1,1,0,0,NaN,NaN,NaN,NaN
1,2020-01-31,1,0,0,0,2,1,0,0,NaN,NaN,NaN,NaN
2,2020-02-09,1,1,0,0,3,2,0,0,NaN,NaN,NaN,NaN
3,2020-02-12,3,0,0,0,6,2,0,0,NaN,NaN,NaN,NaN
4,2020-02-21,1,0,0,0,7,2,0,0,NaN,NaN,NaN,NaN


In [56]:
hosp = bydate[['date', 'ave_7_num_hosp']].set_index('date')

In [57]:
hosp = hosp[6:]

In [58]:
train, test = hosp[:-1], hosp[-1:]

In [59]:
test

,ave_7_num_hosp
date,
2021-06-10,165.571429


In [74]:
model=ARMA(train, order = (7, 2)).fit(disp=False)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [69]:
model.predict(len(train), len(hosp)-1)

2021-06-10    164.650501
Freq: D, dtype: float64

In [75]:
res=[]
for p in range(1, 5):
  for q in range(1, 3):
    model=ARMA(train, order = (p, q)).fit(disp=False)
    res.append((p, q, mse(model.predict(len(train), len(hosp)-1), test)**0.5))

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/

In [76]:
error = [e for p, q, e in res]
best = [res[i] for i, e in enumerate(error) if e == min(error)]

best

[(2, 2, 0.4907650071718308)]

# num_uci

In [130]:
uci = bydate[['date', 'ave_7_num_uci']].set_index('date')

In [131]:
uci = uci[6:]

In [132]:
train, test = uci[:-1], uci[-1:]

In [133]:
test

,ave_7_num_uci
date,
2021-06-10,3.714286


In [143]:
model=ARMA(train, order = (1, 5)).fit(disp=False)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [144]:
model.predict(len(train), len(uci)-1)

2021-06-10    3.642109
Freq: D, dtype: float64

In [145]:
res=[]
for p in range(1, 5):
  for q in range(1, 5):
    model=ARMA(train, order = (p, q)).fit(disp=False)
    res.append((p, q, mse(model.predict(len(train), len(uci)-1), test)**0.5))

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packa

In [146]:
error = [e for p, q, e in res]
best = [res[i] for i, e in enumerate(error) if e == min(error)]

best

[(2, 4, 0.0033038022111191445)]

# num_dead

In [147]:
dead = bydate[['date', 'ave_7_num_dead']].set_index('date')

In [148]:
dead = dead[6:]

In [149]:
train, test = dead[:-1], dead[-1:]

In [150]:
test

,ave_7_num_dead
date,
2021-06-10,3.428571


In [154]:
model=ARMA(train, order = (1, 3)).fit(disp=False)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


In [155]:
model.predict(len(train), len(dead)-1)

2021-06-10    4.007331
Freq: D, dtype: float64

In [161]:
res=[]
for p in range(1, 5):
  for q in range(1, 4):
    model=ARMA(train, order = (p, q)).fit(disp=False)
    res.append((p, q, mse(model.predict(len(train), len(uci)-1), test)**0.5))

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packag

In [162]:
error = [e for p, q, e in res]
best = [res[i] for i, e in enumerate(error) if e == min(error)]

best

[(2, 3, 0.10383019947139394)]